This notebook will assess a variety of ML algorithms in their performance in predicting house prices. We'll be considering lasso and ridge regression, random forests, xgboost and maybe later light gbm. All models will be tuned via Bayesian Optimisation in order to minimise the average 5-fold cross validation score. Note that here specifically we are predicting log house prices and using the RMSE metric so our model evaluation will need to be tailored to that.

First we'll import the data and clean it so that it can be used in a ML friendly format. We'll use a combination of random shuffling and forward filling to get rid of the NAs and convert the strings into dummy variables. We will not be exploring feature creation as there are many features in this data set already. Since exploratory data analysis has already been covered in the other notebook, we won't bother repeating it here.

In [1]:
import pandas as pd
import numpy as np
import time
from scipy.stats import skew
from bayes_opt import BayesianOptimization

In [2]:
#Setting up forward filling function

def fill_nas(df):
    
    #Keeping tack of time
    t0 = time.time()
    
    #Counting NaNs
    na_count = df.isna().sum().sum()
    
    while na_count>0:
        df = df.sample(frac=1)
        df = df.fillna(method='ffill',limit=1)
        na_count = df.isna().sum().sum()

    filled_df = df.sort_index()
    
    #Calculating time taken
    t1 = time.time()
    print(t1-t0)
    
    #Return filled df
    return(filled_df)

In [3]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

#Stripping SalePrice from the training data and combining with the test data
SalePrice = np.log(train['SalePrice'])
train = train.drop('SalePrice',axis=1)
X = pd.concat([train,test], ignore_index=True)

Doing some data pre-processing

In [4]:
#MSSubClass is a categorical variable, so we convert it to a string 
X['MSSubClass'] = X['MSSubClass'].apply(str)

In [5]:
##Dealing with numerics
numerics = X.select_dtypes(exclude='object')

#Filling nas
numerics = fill_nas(numerics)

#log transformation of skewed variables
skews = numerics.apply(lambda x:skew(x.dropna()))
skewed = skews>0.75
skewed_data = numerics[skewed.index[skewed]]
skewed_feats = skewed_data.columns
numerics = numerics.drop(skewed_feats, axis=1)
log_transformed = np.log1p(skewed_data)
numerics = pd.concat([numerics,log_transformed],axis=1)


0.025222301483154297


In [6]:
#Dealing with strings
strings = X.select_dtypes(include='object')

#Converting strings to dummies and joining with numerics
dummies = pd.get_dummies(strings, dummy_na=True)
X = pd.concat([numerics,dummies],axis=1)

#Splitting into train and test data
X_train = X.iloc[:train.shape[0],]
X_test = X.iloc[train.shape[0]:,]
train = pd.concat([SalePrice,X_train],axis=1)
print(X_train.shape, X_test.shape)
print(train.shape, test.shape)

(1460, 348) (1459, 348)
(1460, 349) (1459, 80)


So now that the initial data preprocessing has been done, we are now going to set up cross-validation and Bayesian optimisation procedures for each model. The reason why we use Bayesian Optimisation for hyperparameter tuning is because of the stochastic nature of our objective function. We are randomly sorting the data to better replicate the variability in the dgp when measuring the validation score. While a deterministic optimiser may work, it may not be the best solution if it does not account for the noise.

In [11]:
#Setting up optimisation for Ridge regression model
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

def ridge_score(alpha):
    train_data = train.sample(frac=1)
    X=train_data.iloc[:,1:]
    y=train_data['SalePrice']
    
    scores = cross_val_score(estimator=Ridge(alpha=alpha), X=X, y=y, scoring='neg_mean_squared_error', cv=5)
    return(-np.average(np.sqrt(-scores)))

bounds_ridge = {'alpha': (0,20)}

For the parameter bounds, I have chosen alpha to be between 0 and 20. Using a 0 value will be equivalent to estimation by OLS, any positive values will shrink the parameter estimates towards zero. I do not want to using an upper bound that is too high as it will waste computation power as in practice, optimal values for alpha are relatively small.

In [35]:
#Running Bayes Opt to tune ridge regression

BO = BayesianOptimization(ridge_score, bounds_ridge)
BO.maximize(n_iter=50,alpha=0.0001)

|   iter    |  target   |   alpha   |
-------------------------------------
|  1        | -0.1287   |  9.72     |
|  2        | -0.1265   |  10.92    |
|  3        | -0.1317   |  14.69    |
|  4        | -0.1266   |  7.17     |
|  5        | -0.132    |  1.477    |
|  6        | -0.1297   |  20.0     |
|  7        | -0.1516   |  0.001091 |
|  8        | -0.1265   |  20.0     |
|  9        | -0.1265   |  20.0     |
|  10       | -0.1288   |  20.0     |
|  11       | -0.1283   |  20.0     |
|  12       | -0.1331   |  20.0     |
|  13       | -0.1272   |  20.0     |
|  14       | -0.1315   |  20.0     |
|  15       | -0.1315   |  20.0     |
|  16       | -0.1297   |  20.0     |
|  17       | -8.803e+1 |  0.0      |
|  18       | -0.1283   |  20.0     |
|  19       | -0.1292   |  20.0     |
|  20       | -0.1299   |  20.0     |
|  21       | -0.1426   |  0.001095 |
|  22       | -0.1292   |  20.0     |
|  23       | -0.1301   |  20.0     |
|  24       | -0.1288   |  20.0     |
|  25       

The optimal choice of alpha seems to be around 10.

In [40]:
#Setting up optimisation for Lasso model
from sklearn.linear_model import Lasso

def lasso_score(alpha):
    train_data = train.sample(frac=1)
    X=train_data.iloc[:,1:]
    y=train_data['SalePrice']

    scores = cross_val_score(estimator=Lasso(alpha=alpha, max_iter=10000), X=X, y=y, scoring='neg_mean_squared_error', cv=5)
    return(-np.average(np.sqrt(-scores)))

bounds_lasso = {'alpha': (0.00001,20)}

I'm adjusting the bounds for the lasso as 0 values for regularisation cause convergence problems.

In [41]:
#Running Bayes Opt to tune Lasso regression

BO = BayesianOptimization(lasso_score, bounds_lasso)
BO.maximize(n_iter=50,alpha=0.0001)

|   iter    |  target   |   alpha   |
-------------------------------------
|  1        | -0.305    |  7.299    |
|  2        | -0.3066   |  9.549    |
|  3        | -0.3148   |  17.51    |
|  4        | -0.3044   |  5.857    |
|  5        | -0.316    |  19.1     |
|  6        | -0.1356   |  1.006e-0 |
|  7        | -0.1363   |  1e-05    |
|  8        | -0.132    |  1e-05    |
|  9        | -0.132    |  1e-05    |
|  10       | -0.132    |  1e-05    |
|  11       | -0.132    |  1e-05    |
|  12       | -0.1363   |  1e-05    |
|  13       | -0.1418   |  1e-05    |
|  14       | -0.1418   |  1e-05    |
|  15       | -0.1418   |  1e-05    |
|  16       | -0.1418   |  1e-05    |
|  17       | -0.1418   |  1e-05    |
|  18       | -0.1418   |  1e-05    |
|  19       | -0.1418   |  1e-05    |
|  20       | -0.1418   |  1e-05    |
|  21       | -0.1418   |  1e-05    |
|  22       | -0.1418   |  1e-05    |
|  23       | -0.1418   |  1e-05    |
|  24       | -0.1418   |  1e-05    |
|  25       

I'm quite suspicious about the behaviour of the Bayes optimiser in this case, it cannot seem to explore other values than the lower bound. While I could change the lower bound, it will cause convergence issues and will require an increased number of iterations. I can handpick a value for alpha that does noticeably better.

In [44]:
lasso_score(0.0001)

-0.12857380432989013

At this point I'm not going to try delve into the workings of optimiser as I don't think it will be very fruitful. Instead I'm going to try using a different optimiser to see if we get better convergence.

In [48]:
from scipy.optimize import minimize

minimize(lasso_score, x0=1, bounds = ((0.00001,20),), tol=0.00001)

      fun: -0.2677542767110079
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([6881.69102627])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 50
      nit: 2
   status: 0
  success: True
        x: array([0.99999875])

This also performs quite poorly. I think it gets thrown off by the randomness of the objective function. It does find a good optimum, but only if I put in an optimal starting point!

In [49]:
minimize(lasso_score, x0=0.001, bounds = ((0.00001,20),), tol=0.00001)

      fun: -0.12544040196865947
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([-196060.25283075])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 32
      nit: 2
   status: 0
  success: True
        x: array([0.00097682])

In [ ]:
With a good choice of alpha, the lasso marginally outpeforms the ridge regression.

In [9]:
#Creating optimizer for a Random Forests regressor
from sklearn.ensemble import RandomForestRegressor as RF

def RF_score(n_estimators,max_depth,min_samples_split,min_samples_leaf,max_features):
    
    #Contraining hyperparameters to be converted to integers (e.g. number of decision trees can't be continuous!)
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    min_samples_split = int(min_samples_split)
    min_samples_leaf = int(min_samples_leaf)
    max_features = int(max_features)
    
    assert type(n_estimators) == int
    assert type(max_depth) == int
    assert type(min_samples_split) == int
    assert type(min_samples_leaf) == int
    assert type(max_features) == int
    
    train_data = train.sample(frac=1)
    X=train_data.iloc[:,1:]
    y=train_data['SalePrice']

    scores = cross_val_score(
        estimator=RF(
                    n_estimators=n_estimators, 
                    max_depth=max_depth, 
                    min_samples_split=min_samples_split,
                    min_samples_leaf = min_samples_leaf,
                    max_features = max_features),
    X=X, y=y, scoring='neg_mean_squared_error', cv=5)
    return(-np.average(np.sqrt(-scores)))

bounds_RF = {
    'n_estimators': (1,3000),
    'max_depth': (1,100),
    'min_samples_split': (2,200),
    'min_samples_leaf': (1,200),
    'max_features': (1,290)
}

In [12]:
#Running Bayes Opt to tune Rando Forests regression

BO = BayesianOptimization(RF_score, bounds_RF)
BO.maximize(n_iter=50,alpha=0.0001)

|   iter    |  target   | max_depth | max_fe... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------
|  1        | -0.2525   |  83.77    |  47.77    |  165.1    |  117.2    |  44.25    |
|  2        | -0.2654   |  63.19    |  22.49    |  145.3    |  133.5    |  2.303e+0 |
|  3        | -0.2111   |  65.32    |  236.2    |  107.6    |  143.9    |  946.7    |
|  4        | -0.2505   |  6.298    |  263.0    |  158.0    |  167.8    |  1.276e+0 |
|  5        | -0.2573   |  6.909    |  273.4    |  197.2    |  141.0    |  1.167e+0 |
|  6        | -0.2822   |  1.0      |  290.0    |  1.0      |  41.37    |  3e+03    |
|  7        | -0.1475   |  99.28    |  31.63    |  3.139    |  9.578    |  1.143e+0 |
|  8        | -0.3804   |  1.0      |  1.0      |  1.0      |  2.0      |  1.114e+0 |
|  9        | -0.1405   |  100.0    |  290.0    |  1.0      |  2.0      |  1.918e+0 |
|  10       | -0.2284   |  100.0    |  290.0    |  1.0

In [62]:
#Creating optimiser for xgboost, note that doing it this way isn't entirely necessary
#xgboost already contains the inbuilt functionality to do so, but I want to be consistent in my approach
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb

def xgb_score(eta, max_depth, gamma):
    
    #Contraining hyperparameters to be converted to integers (e.g. number of decision trees can't be continuous!)
    #n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    #min_samples_split = int(min_samples_split)
    #min_samples_leaf = int(min_samples_leaf)
    #max_features = int(max_features)
    
    #assert type(n_estimators) == int
    assert type(max_depth) == int
    #assert type(min_samples_split) == int
    #assert type(min_samples_leaf) == int
    #assert type(max_features) == int
    
    train_data = train.sample(frac=1)
    X=train_data.iloc[:,1:]
    #X = xgb.DMatrix(X)
    y=np.array(train_data['SalePrice'])
    

    xgb_model = xgb.XGBRegressor(learning_rate=0.1, 
                                 max_depth=max_depth, 
                                 min_split_loss=gamma, 
                                 objective="reg:squarederror")
    
    scores = cross_val_score(estimator=xgb_model, X=X, y=y, scoring='neg_mean_squared_error', cv=5)
    return(-np.average(np.sqrt(-scores)))

bounds_xgb = {'eta': (0.08,0.12),'max_depth':(5,15),'gamma':(0,0.1)}

In [8]:
#Running Bayes Opt to tune Randof Forests regression

BO = BayesianOptimization(xgb_score, bound_xgb)
BO.maximize(n_iter=50,alpha=0.0001)

NameError: name 'xgb_score' is not defined

So now that we've tuned the hyperparameters using Bayes opt and gotten a good feel of how the models perform using different parameter sets, it's pretty easy to see that xgboost performs the best. As a final evaluation, I'll put in some paramter values to evaluate our models concurrently and then pick from there.

In [19]:
print(
    #lasso_score(0.001),
    #ridge_score(10),
    RF_score(n_estimators=20, 
                    max_depth=100, 
                    min_samples_split=2,
                    min_samples_leaf = 1,
                    max_features = 130)
    #xgb_score(eta=0.1,max_depth=10,gamma=0)
    )

-0.14174324638541208


As noted, xgboost clearly outperforms all other models. So we'll use that configuration to make our test predictions and submit on Kaggle.

In [156]:
model_xgb = xgb.XGBRegressor(learning_rate=0.1, max_depth=10, min_split_loss=0, objective="reg:squarederror")
model_xgb.fit(X_train,SalePrice)
xgb_preds = np.expm1(model_xgb.predict(X_test))


#Submission csv
submission = pd.read_csv('Data/sample_submission.csv')
submission['SalePrice'] = xgb_preds
submission.to_csv('Data/submission.csv',index=False)

/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Well! Turns out we did rather poorly despite all the work we put in, we got a public score of 0.14504 placing us in the bottom 50% :(.

In [52]:
model_lasso = Lasso(alpha=0.001)
model_lasso.fit(X_train,SalePrice)
lasso_preds = np.expm1(model_lasso.predict(X_test))

#Submission csv
submission = pd.read_csv('Data/sample_submission.csv')
submission['SalePrice'] = lasso_preds
submission.to_csv('Data/submission.csv',index=False)